In [1]:
import pandas as pd

In [2]:
# from predicted_hosts_EDA.ipynb
iphop_df = pd.read_csv('/fs/ess/PAS1117/riddell26/Grantham_Bioreactor/03-predict-hosts/results/iphop_custom/Host_prediction_to_genome_m90.csv')
iphop_df.head()

,Virus,Host genome,Host taxonomy,Main method,Confidence score,Additional methods
0,contig_1000262,RS_GCF_900167265.1,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,iPHoP-RF,92.8,NaN
1,contig_1000294|provirus_2_36746,RS_GCF_005938625.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.4,iPHoP-RF;73.50
2,contig_1000294|provirus_2_36746,RS_GCF_012986595.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.3,iPHoP-RF;59.20
3,contig_1000294|provirus_2_36746,RS_GCF_000633255.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.1,iPHoP-RF;80.00
4,contig_1000294|provirus_2_36746,RS_GCF_002890915.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,91.8,CRISPR;58.50


In [3]:
# load in updated host classification from GTDB-classify
arch = pd.read_csv('/fs/ess/PAS1117/riddell26/Grantham_Bioreactor/03-predict-hosts/data/gtdb_classify_wf_for_virmatcher/gtdbtk.ar122.summary.tsv', sep='\t')
arch['Host genome'] = arch['user_genome']
arch['Host taxonomy'] = arch['classification']
arch = arch[['Host genome', 'Host taxonomy']]

bac = pd.read_csv('/fs/ess/PAS1117/riddell26/Grantham_Bioreactor/03-predict-hosts/data/gtdb_classify_wf_for_virmatcher/gtdbtk.bac120.summary.tsv', sep='\t')
bac['Host genome'] = bac['user_genome']
bac['Host taxonomy'] = bac['classification']
bac = bac[['Host genome', 'Host taxonomy']]

host_taxonomy = pd.concat([arch, bac])
host_taxonomy.head()

,Host genome,Host taxonomy
0,Add_20100900_E3D_5,d__Archaea;p__Methanobacteriota;c__Methanobact...
1,Add_20110600_P1M_11,d__Archaea;p__Thermoproteota;c__Nitrososphaeri...
2,Add_20110700_S2D_21,d__Archaea;p__Methanobacteriota;c__Methanobact...
3,Add_20110800_E1S_4,d__Archaea;p__Halobacteriota;c__Methanosarcini...
4,Add_20110800_E1S_5,d__Archaea;p__Halobacteriota;c__Methanomicrobi...


In [4]:
# replace host taxonomy for hosts from the custom iphop database

# Merge to get updated taxonomy only for matching 'Add_' host genomes
iphop_df = iphop_df.merge(host_taxonomy, on="Host genome", how="left", suffixes=("", "_new"))

# Replace 'Host taxonomy' where 'Host genome' starts with 'Add_'
iphop_df["Host taxonomy"] = iphop_df.apply(
    lambda row: row["Host taxonomy_new"] if row["Host genome"].startswith("Add_") and pd.notna(row["Host taxonomy_new"]) else row["Host taxonomy"],
    axis=1
)

# Drop the extra merged column
iphop_df.drop(columns=["Host taxonomy_new"], inplace=True)
iphop_df.head()

,Virus,Host genome,Host taxonomy,Main method,Confidence score,Additional methods
0,contig_1000262,RS_GCF_900167265.1,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,iPHoP-RF,92.8,NaN
1,contig_1000294|provirus_2_36746,RS_GCF_005938625.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.4,iPHoP-RF;73.50
2,contig_1000294|provirus_2_36746,RS_GCF_012986595.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.3,iPHoP-RF;59.20
3,contig_1000294|provirus_2_36746,RS_GCF_000633255.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.1,iPHoP-RF;80.00
4,contig_1000294|provirus_2_36746,RS_GCF_002890915.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,91.8,CRISPR;58.50


In [17]:
# How many vOTUs had an iPHoP prediction?=
print('number of vOTUs from reference dataset with a host prediction: ' + str(iphop_df.Virus.nunique()))

number of vOTUs from reference dataset with a host prediction: 3187


In [16]:
# How many transciptionally active vOTUs had an iPHoP prediction?
with open('/fs/scratch/Sullivan_Lab/JamesR/Grantham_Bioreactor/02-get-relative-abundance/results/metaT/active_1_gene_per_10kb_vOTUs.txt', 'r') as f:
    active_vOTUs = [i.strip() for i in f.readlines()]

num_active_predictions = iphop_df.loc[iphop_df['Virus'].isin(active_vOTUs)].Virus.nunique()
print('number of active vOTUs with a host prediction: ' + str(num_active_predictions))

number of active vOTUs with a host prediction: 2055


In [23]:
# number of iphop predictions with a Stordalen host prediction
print('number of vOTUs from reference dataset with a host prediction: ' + str(iphop_df.loc[iphop_df['Host genome'].str.startswith('Add_')].Virus.nunique()))

number of vOTUs from reference dataset with a host prediction: 2000


In [25]:
# number of iphop predictions with a Stordalen host prediction
print('number of active vOTUs from reference dataset with a host prediction: ' + str(iphop_df
                                                                                     .loc[
                                                                                     (iphop_df['Host genome'].str.startswith('Add_'))
                                                                                     & (iphop_df['Virus'].isin(active_vOTUs)) 
                                                                                      ].Virus.nunique()))

number of active vOTUs from reference dataset with a host prediction: 1343


In [34]:
# number of active vOTUs with an active host
MAG_abundance = pd.read_csv('/fs/ess/PAS1117/riddell26/Grantham_Bioreactor/02-get-relative-abundance/results/meancov/metaG/MAGs/metaG_MAG_mean_97_75_75.tsv', sep='\t')
present_MAGs = MAG_abundance.loc[MAG_abundance.iloc[:,1:].sum(axis=1) > 0]

In [38]:
present_MAGs_list = ['Add_' + i for i in present_MAGs['Genome']]

In [39]:
active_vOTUs_present_MAGs = (iphop_df
     .loc[
     (iphop_df['Host genome'].isin(present_MAGs_list))
     & (iphop_df['Virus'].isin(active_vOTUs)) 
      ]
)

In [41]:
len(active_vOTUs_present_MAGs)

1864

In [42]:
# build virus-host matrix with relevant metadata


In [48]:
# add provirus
vOTU_metadata = pd.read_csv('/fs/scratch/Sullivan_Lab/JamesR/Grantham_Bioreactor/01-build-vOTU-database/results/vOTUs/vOTUs_filtered_metadata.tsv', sep='\t')
is_provirus = vOTU_metadata[['vOTU', 'is_provirus_aggregated']].drop_duplicates()

/tmp/slurmtmp.793313/ipykernel_2622651/2136389251.py:2: DtypeWarning: Columns (3,23) have mixed types. Specify dtype option on import or set low_memory=False.
  vOTU_metadata = pd.read_csv('/fs/scratch/Sullivan_Lab/JamesR/Grantham_Bioreactor/01-build-vOTU-database/results/vOTUs/vOTUs_filtered_metadata.tsv', sep='\t')


In [50]:
iphop_df = iphop_df.rename(columns={'Virus': 'vOTU'})

In [51]:
iphop_df = iphop_df.merge(is_provirus, on='vOTU', how='left')

In [53]:
iphop_df

,vOTU,Host genome,Host taxonomy,Main method,Confidence score,Additional methods,is_provirus_aggregated
0,contig_1000262,RS_GCF_900167265.1,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,iPHoP-RF,92.8,NaN,False
1,contig_1000294|provirus_2_36746,RS_GCF_005938625.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.4,iPHoP-RF;73.50,True
2,contig_1000294|provirus_2_36746,RS_GCF_012986595.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.3,iPHoP-RF;59.20,True
3,contig_1000294|provirus_2_36746,RS_GCF_000633255.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.1,iPHoP-RF;80.00,True
4,contig_1000294|provirus_2_36746,RS_GCF_002890915.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,91.8,CRISPR;58.50,True
...,...,...,...,...,...,...,...
23194,contig_999988,RS_GCF_002843605.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,90.5,blast;89.40,False
23195,contig_999988,RS_GCF_013385985.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,90.5,NaN,False
23196,contig_999988,RS_GCF_004519405.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,90.4,iPHoP-RF;89.80,False
23197,contig_999988,RS_GCF_000503215.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,90.1,NaN,False


In [54]:
iphop_df[['K', 'P', 'C', 'O', 'F', 'G', 'S']] = iphop_df['Host taxonomy'].str.split(';', expand=True)

In [55]:
# Define the order of columns from highest to lowest resolution
taxonomy_columns = ['G', 'F', 'O', 'C', 'P']

# Create a new column with the highest resolution taxonomic value
def get_highest_resolution(row):
    for col in taxonomy_columns:
        if row[col] != f'{col.lower()}__':  # Check if the value is not the default prefix
            return row[col]
    return np.nan  # If no valid value is found, return NaN

iphop_df['highest_host_tax_rank'] = iphop_df.apply(get_highest_resolution, axis=1)

In [63]:
iphop_df['vOTU_is_active'] = iphop_df['vOTU'].isin(active_vOTUs)
iphop_df['MAG_is_present'] = iphop_df['Host genome'].isin(present_MAGs_list)
iphop_df['MAG_is_stordalen'] = iphop_df['Host genome'].str.startswith('Add_')

In [64]:
iphop_df.head()

,vOTU,Host genome,Host taxonomy,Main method,Confidence score,Additional methods,is_provirus_aggregated,K,P,C,O,F,G,S,highest_host_tax_rank,vOTU_is_active,MAG_is_present,MAG_is_stordalen
0,contig_1000262,RS_GCF_900167265.1,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,iPHoP-RF,92.8,NaN,False,d__Bacteria,p__Bacillota_A,c__Clostridia,o__Clostridiales,f__Clostridiaceae,g__Clostridium_G,s__Clostridium_G tetani,g__Clostridium_G,True,False,False
1,contig_1000294|provirus_2_36746,RS_GCF_005938625.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.4,iPHoP-RF;73.50,True,d__Bacteria,p__Pseudomonadota,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas_E,s__Pseudomonas_E fluorescens_BD,g__Pseudomonas_E,True,False,False
2,contig_1000294|provirus_2_36746,RS_GCF_012986595.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.3,iPHoP-RF;59.20,True,d__Bacteria,p__Pseudomonadota,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas_E,s__Pseudomonas_E koreensis,g__Pseudomonas_E,True,False,False
3,contig_1000294|provirus_2_36746,RS_GCF_000633255.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,blast,92.1,iPHoP-RF;80.00,True,d__Bacteria,p__Pseudomonadota,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas_E,s__Pseudomonas_E sp000633255,g__Pseudomonas_E,True,False,False
4,contig_1000294|provirus_2_36746,RS_GCF_002890915.1,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,91.8,CRISPR;58.50,True,d__Bacteria,p__Pseudomonadota,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Stutzerimonas,s__Stutzerimonas stutzeri_AF,g__Stutzerimonas,True,False,False


In [65]:
active_vOTUs_present_MAGs.to_csv('/fs/ess/PAS1117/riddell26/Grantham_Bioreactor/figures/data/vOTUs_MAGs_for_cytoscape.tsv', sep='\t', index=False)

In [66]:
active_vOTUs_present_MAGs = (iphop_df
     .loc[
            (iphop_df['vOTU_is_active'] == True) 
          & (iphop_df['MAG_is_present'] == True)
         ]
)

In [67]:
active_vOTUs_present_MAGs

,vOTU,Host genome,Host taxonomy,Main method,Confidence score,Additional methods,is_provirus_aggregated,K,P,C,O,F,G,S,highest_host_tax_rank,vOTU_is_active,MAG_is_present,MAG_is_stordalen
13,contig_1000567,Add_STM_0716_E_M_E026_E050_A_bin.98,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,blast,92.1,iPHoP-RF;70.90,False,d__Bacteria,p__Bacillota_A,c__Clostridia,o__Clostridiales,f__Clostridiaceae,g__Clostridium,s__,g__Clostridium,True,True,True
16,contig_1000567,Add_Bog.Labelled.R1.T1.F1.JGI.BFC.metaspades_b...,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,blast,91.0,NaN,False,d__Bacteria,p__Bacillota_A,c__Clostridia,o__Clostridiales,f__Clostridiaceae,g__Clostridium,s__,g__Clostridium,True,True,True
18,contig_1000567,Add_STM_0716_E_M_E026_A_bin.32,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,blast,90.5,NaN,False,d__Bacteria,p__Bacillota_A,c__Clostridia,o__Clostridiales,f__Clostridiaceae,g__Clostridium,s__,g__Clostridium,True,True,True
20,contig_1000567,Add_STM_0716_E_M_E026_E030_E034_D_multi_bin.46,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,blast,90.2,NaN,False,d__Bacteria,p__Bacillota_A,c__Clostridia,o__Clostridiales,f__Clostridiaceae,g__Clostridium,s__,g__Clostridium,True,True,True
21,contig_1000706,Add_STM_0716_E_M_E069_E065_A_bin.5,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Cl...,blast,94.2,iPHoP-RF;77.80,False,d__Bacteria,p__Bacillota_A,c__Clostridia,o__Clostridiales,f__Clostridiaceae,g__Clostridium,s__,g__Clostridium,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23000,contig_992236,Add_STM_0716_E_M_E030_E054_E065_D_bin.5,d__Bacteria;p__Acidobacteriota;c__Terriglobia;...,blast,95.0,iPHoP-RF;83.10,False,d__Bacteria,p__Acidobacteriota,c__Terriglobia,o__Bryobacterales,f__Bryobacteraceae,g__Sulfopaludibacter,s__,g__Sulfopaludibacter,True,True,True
23077,contig_995332,Add_Fen.Labelled.R1.T2.F4.trimmomatic.metaspad...,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,iPHoP-RF,91.8,NaN,False,d__Bacteria,p__Pseudomonadota,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas_E,s__Pseudomonas_E mohnii,g__Pseudomonas_E,True,True,True
23105,contig_996080,Add_PMBY01,d__Bacteria;p__Myxococcota;c__Polyangia;o__Fen...,blast,91.7,iPHoP-RF;60.60,False,d__Bacteria,p__Myxococcota,c__Polyangia,o__Fen-1088,f__Fen-1088,g__Fen-1137,s__Fen-1137 sp003153875,g__Fen-1137,True,True,True
23107,contig_996080,Add_PMBS01,d__Bacteria;p__Myxococcota;c__Polyangia;o__Fen...,blast,91.3,iPHoP-RF;67.70,False,d__Bacteria,p__Myxococcota,c__Polyangia,o__Fen-1088,f__Fen-1088,g__Fen-1137,s__Fen-1137 sp003153015,g__Fen-1137,True,True,True


In [68]:
active_vOTUs_present_MAGs.to_csv('/fs/ess/PAS1117/riddell26/Grantham_Bioreactor/figures/data/active_vOTUs_present_MAGs_for_cytoscape.tsv', sep='\t', index=False)